In [1]:
import os, sys,time

import numpy as np
import pandas as pd
import logging

import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)
 
DATA_DIR = "/var/nvidia/vishal/A3NCF-master/python/data/"



In [2]:
tf.reset_default_graph()

In [3]:
def read_feature(dataset, dimension, name):
    """ read user and item features into dictionary."""
    entity_feature = {}
    with open(os.path.join(DATA_DIR, dataset+"."+str(dimension)+"."+name+".theta")) as entity_file:
        for entity_info in entity_file:
            line = entity_info.strip().split(',')
            entity = [float(i) for i in line[1:]]
            entity_feature[int(line[0])] = np.array(entity, dtype=np.float32)
    return entity_feature

In [4]:
def read_data(dataset='Baby', dimension=5, batch_size=128, is_training=True):
    user_feature = read_feature(dataset, dimension, 'user')
    item_feature = read_feature(dataset, dimension, 'item')
    user_size = len(user_feature)
    item_size = len(item_feature)

    split = 'train' if is_training else 'test'
    data = pd.read_csv(os.path.join(DATA_DIR, dataset+".{}.dat".format(split)), sep='\t', header=None,names=['user', 'item', 'rating'],dtype={'rating': np.float32})

    ######################## ADD FEATURES #####################
    user_feature_col, item_feature_col = [], []
    for i in range(len(data)):
        user_feature_col.append(user_feature[data['user'][i]])
        item_feature_col.append(item_feature[data['item'][i]])
    # data['user_feature'] = user_feature_col
    # data['item_feature'] = item_feature_col

    data_dict = dict([('user', data['user'].values),
                ('item', data['item'].values),
                ('rating', data['rating'].values),
                ('user_feature', np.array(user_feature_col)),
                ('item_feature', np.array(item_feature_col))])
    dataset = tf.data.Dataset.from_tensor_slices(data_dict)
    if is_training:
        dataset = dataset.shuffle(10000)
    dataset = dataset.batch(batch_size)

    return dataset, user_size, item_size

<img src="files/A3ncf.png">

In [5]:
class AttNCF(object):
    def __init__(self, user_size, item_size, embed_size, iterator,
                 activation_func, lr, optim, regularizer, dropout, is_training):
        self.user_size = user_size
        self.item_size = item_size
        self.embed_size = embed_size
        self.iterator = iterator
        self.activation_func = activation_func
        self.lr = lr
        self.optim = optim
        self.regularizer_rate = regularizer
        self.dropout = dropout
        self.is_training = is_training

    def get_data(self):
        """ Obtain the input data from tensorflow iterator."""
        sample = self.iterator.get_next()

        self.user = sample['user']
        self.item = sample['item']
        self.user_feature = sample['user_feature']
        self.item_feature = sample['item_feature']
        self.rating = sample['rating']

    def inference(self):
        self.regularizer = tf.contrib.layers.l2_regularizer(self.regularizer_rate)
        
        if self.activation_func == 'ReLU':
            self.activation_func = tf.nn.relu
        elif self.activation_func == 'Leaky_ReLU':
            self.activation_func = tf.nn.leaky_relu
        elif self.activation_func == 'ELU':
            self.activation_func = tf.nn.elu

        if self.optim == 'SGD':
            self.optimizer = tf.train.GradientDescentOptimizer(self.lr, name='SGD')
        elif self.optim == 'RMSProp':
            self.optimizer = tf.train.RMSPropOptimizer(self.lr, decay=0.9, momentum=0.0, name='RMSProp')
        elif self.optim == 'Adam':
            self.optimizer = tf.train.AdamOptimizer(self.lr, name='Adam')

    def create_model(self):
        """ Create model from scratch. """
        with tf.name_scope("input"):
            self.user_embedding = tf.get_variable("user_embed", [self.user_size, self.embed_size], dtype=tf.float32)
            self.item_embedding = tf.get_variable("item_embed", [self.item_size, self.embed_size], dtype=tf.float32)
            self.user_embed = tf.nn.embedding_lookup(self.user_embedding, self.user)
            self.item_embed = tf.nn.embedding_lookup(self.user_embedding, self.item)
        with tf.name_scope("fusion"):
            self.user_fusion_add = self.user_embed + self.user_feature
            self.item_fusion_add = self.item_embed + self.item_feature

            self.user_fusion = tf.layers.dense(inputs=self.user_fusion_add,units=self.embed_size,activation=self.activation_func,
                                               kernel_regularizer=self.regularizer,name='user_fusion')
            self.item_fusion = tf.layers.dense(inputs=self.item_fusion_add,units=self.embed_size,activation=self.activation_func,
                                               kernel_regularizer=self.regularizer,name='item_fusion')

        with tf.name_scope("attention"):
            self.feature_all = tf.concat([self.user_feature, self.item_feature,self.user_fusion, self.item_fusion], -1)
            self.att_layer1 = tf.layers.dense(inputs=self.feature_all,units=1,activation=self.activation_func,
                                              kernel_regularizer=self.regularizer,name='att_layer1')
            self.att_layer2 = tf.layers.dense(inputs=self.att_layer1,units=self.embed_size,activation=self.activation_func,
                                              kernel_regularizer=self.regularizer,name='att_layer2')
            self.att_weights = tf.nn.softmax(self.att_layer2, axis=-1, name='att_softmax')

        with tf.name_scope("prediction"):
            self.interact = self.att_weights*self.user_fusion*self.item_fusion
            self.interact1 = tf.layers.dense(inputs=self.interact,units=self.embed_size,activation=self.activation_func,
                                             kernel_regularizer=self.regularizer,name='interact1')
            self.interact1 = tf.nn.dropout(self.interact1, self.dropout)
            self.prediction = tf.layers.dense(inputs=self.interact,units=1,activation=None,
                                              kernel_regularizer=self.regularizer,name='prediction')
            self.prediction = tf.reshape(self.prediction, [-1])

    def loss_func(self):
        reg = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        reg_loss = tf.contrib.layers.apply_regularization(self.regularizer, reg)
        mse_loss = tf.losses.mean_squared_error(self.rating, self.prediction)
        self.loss =  mse_loss

    def optimization(self):
        with tf.name_scope("optimization"):
            self.optim = self.optimizer.minimize(self.loss)

    def eval(self):
        """ Evaluate each sample."""
        self.se = tf.square(self.rating - self.prediction)


    def build(self):
        self.get_data()
        self.inference()
        self.create_model()
        self.loss_func()
        self.optimization()
        self.eval()
        self.saver = tf.train.Saver(tf.global_variables())

    def step(self, sess, step):
        """ Train the model step by step. """
        if self.is_training:
            loss, optim,se = sess.run([self.loss, self.optim,self.se])
            return loss,np.sqrt(np.mean(se))
        else:
            se = sess.run([self.se])[0]
            return se



In [6]:
batch_size = 256
embed_size = 5
epochs = 20
dataset = 'Baby'
model_dir = './AttNCF9.5/'
optim = 'Adam'
activation = 'ReLU'
lr = 0.002
dropout = 0.5
l2_rate = 0.001
config = tf.ConfigProto()
config.gpu_options.allow_growth = True


In [7]:
train_data, user_size, item_size = read_data(dataset, embed_size, batch_size, True)
test_data, user_size, item_size = read_data(dataset, embed_size, batch_size, False)
iterator = tf.data.Iterator.from_structure(tf.compat.v1.data.get_output_types(train_data), tf.compat.v1.data.get_output_shapes(train_data))



In [8]:
with tf.Session(config=config) as sess:
    model = AttNCF(user_size, item_size, embed_size,iterator, activation, lr, optim,l2_rate, dropout, is_training=True)
    model.build()
    ckpt = tf.train.get_checkpoint_state(model_dir)
    if ckpt:
        print("Reading model parameters from %s".format(ckpt.model_checkpoint_path))
        model.saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        print("Creating model with fresh parameters.")
        sess.run(tf.global_variables_initializer())
    count = 0
    for epoch in range(epochs):
        sess.run(model.iterator.make_initializer(train_data))
        model.is_training = True
        start_time = time.time()
        try:
            batch_no= 0
            while True:
                loss,se = model.step(sess, count)
#                 if(batch_no%100==0):
#                     print(batch_no,loss,se)
                batch_no+=1
                count += 1
        except tf.errors.OutOfRangeError:
            pass
        
        sess.run(model.iterator.make_initializer(test_data))
        model.is_training = False
        RMSE = np.array([], dtype=np.float32)
        try:
            while True:
                se = model.step(sess, None)
                RMSE = np.append(RMSE, se)
        except tf.errors.OutOfRangeError:
            print("For epoch: ",epoch," Test RMSE is ",np.sqrt(RMSE.mean()))
            
        
        checkpoint_path = os.path.join(model_dir, "AttNCF.ckpt")
        model.saver.save(sess, checkpoint_path)
        

Reading model parameters from %s
('For epoch: ', 0, ' Test RMSE is ', 1.1518992)
('For epoch: ', 1, ' Test RMSE is ', 1.1608667)
('For epoch: ', 2, ' Test RMSE is ', 1.1688697)
('For epoch: ', 3, ' Test RMSE is ', 1.173174)
('For epoch: ', 4, ' Test RMSE is ', 1.1713037)
('For epoch: ', 5, ' Test RMSE is ', 1.1829746)
('For epoch: ', 6, ' Test RMSE is ', 1.1860554)
('For epoch: ', 7, ' Test RMSE is ', 1.1946323)
('For epoch: ', 8, ' Test RMSE is ', 1.1950208)
('For epoch: ', 9, ' Test RMSE is ', 1.1992342)
('For epoch: ', 10, ' Test RMSE is ', 1.2151142)
('For epoch: ', 11, ' Test RMSE is ', 1.2134184)
('For epoch: ', 12, ' Test RMSE is ', 1.216163)
('For epoch: ', 13, ' Test RMSE is ', 1.2192904)
('For epoch: ', 14, ' Test RMSE is ', 1.2261074)
('For epoch: ', 15, ' Test RMSE is ', 1.2359293)
('For epoch: ', 16, ' Test RMSE is ', 1.2204726)
('For epoch: ', 17, ' Test RMSE is ', 1.2364405)
('For epoch: ', 18, ' Test RMSE is ', 1.2456203)
('For epoch: ', 19, ' Test RMSE is ', 1.2436578)

# 90% of this code is from https://github.com/hustlingchen/A3NCF, https://github.com/guoyang9/A3NCF